In [1]:
import pyspark
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.appName("SuperMarketSalesForecast").getOrCreate()

# Read the CSV files and create Spark DataFrames
train_df = spark.read.csv('s3://sm-studio-bucket/SuperMarketSalesDataInput/train.csv', header=True, inferSchema=True)
store_df = spark.read.csv('s3://sm-studio-bucket/SuperMarketSalesDataInput/stores.csv', header=True, inferSchema=True)
feature_df = spark.read.csv('s3://sm-studio-bucket/SuperMarketSalesDataInput/features.csv', header=True, inferSchema=True)
test_df = spark.read.csv('s3://sm-studio-bucket/SuperMarketSalesDataInput/test.csv', header=True, inferSchema=True)

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1715630291204_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
train_df.printSchema()
# Print the first 5 rows of each DataFrame
train_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Store: integer (nullable = true)
 |-- Dept: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- Weekly_Sales: double (nullable = true)
 |-- IsHoliday: boolean (nullable = true)

+-----+----+----------+------------+---------+
|Store|Dept|      Date|Weekly_Sales|IsHoliday|
+-----+----+----------+------------+---------+
|    1|   1|2010-02-05|     24924.5|    false|
|    1|   1|2010-02-12|    46039.49|     true|
|    1|   1|2010-02-19|    41595.55|    false|
|    1|   1|2010-02-26|    19403.54|    false|
|    1|   1|2010-03-05|     21827.9|    false|
+-----+----+----------+------------+---------+
only showing top 5 rows

In [7]:
store_df.printSchema()
store_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Store: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Size: integer (nullable = true)

+-----+----+------+
|Store|Type|  Size|
+-----+----+------+
|    1|   A|151315|
|    2|   A|202307|
|    3|   B| 37392|
|    4|   A|205863|
|    5|   B| 34875|
+-----+----+------+
only showing top 5 rows

In [8]:
feature_df.printSchema()
feature_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Store: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- Temperature: double (nullable = true)
 |-- Fuel_Price: double (nullable = true)
 |-- MarkDown1: string (nullable = true)
 |-- MarkDown2: string (nullable = true)
 |-- MarkDown3: string (nullable = true)
 |-- MarkDown4: string (nullable = true)
 |-- MarkDown5: string (nullable = true)
 |-- CPI: string (nullable = true)
 |-- Unemployment: string (nullable = true)
 |-- IsHoliday: boolean (nullable = true)

+-----+----------+-----------+----------+---------+---------+---------+---------+---------+-----------+------------+---------+
|Store|      Date|Temperature|Fuel_Price|MarkDown1|MarkDown2|MarkDown3|MarkDown4|MarkDown5|        CPI|Unemployment|IsHoliday|
+-----+----------+-----------+----------+---------+---------+---------+---------+---------+-----------+------------+---------+
|    1|2010-02-05|      42.31|     2.572|       NA|       NA|       NA|       NA|       NA|211.0963582|       8.106|    false|
|  

In [9]:
test_df.printSchema()
test_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Store: integer (nullable = true)
 |-- Dept: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- IsHoliday: boolean (nullable = true)

+-----+----+----------+---------+
|Store|Dept|      Date|IsHoliday|
+-----+----+----------+---------+
|    1|   1|2012-11-02|    false|
|    1|   1|2012-11-09|    false|
|    1|   1|2012-11-16|    false|
|    1|   1|2012-11-23|     true|
|    1|   1|2012-11-30|    false|
+-----+----+----------+---------+
only showing top 5 rows

In [10]:
# install numpy using sc for current spark context
sc.install_pypi_package("numpy")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
from pyspark.sql.functions import col, sum as spark_sum

# Count the null values in each column
null_counts = feature_df.select(*[
    spark_sum(col(column_name).isNull().cast("int")).alias(column_name)
    for column_name in feature_df.columns
])

# Show the result
null_counts.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+----+-----------+----------+---------+---------+---------+---------+---------+---+------------+---------+
|Store|Date|Temperature|Fuel_Price|MarkDown1|MarkDown2|MarkDown3|MarkDown4|MarkDown5|CPI|Unemployment|IsHoliday|
+-----+----+-----------+----------+---------+---------+---------+---------+---------+---+------------+---------+
|    0|   0|          0|         0|        0|        0|        0|        0|        0|  0|           0|        0|
+-----+----+-----------+----------+---------+---------+---------+---------+---------+---+------------+---------+

In [12]:
# Drop the columns with  null values
columns_to_drop = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']

# Drop the specified columns
feature_df = feature_df.drop(*columns_to_drop)
feature_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+----------+-----------+----------+-----------+------------+---------+
|Store|      Date|Temperature|Fuel_Price|        CPI|Unemployment|IsHoliday|
+-----+----------+-----------+----------+-----------+------------+---------+
|    1|2010-02-05|      42.31|     2.572|211.0963582|       8.106|    false|
|    1|2010-02-12|      38.51|     2.548|211.2421698|       8.106|     true|
|    1|2010-02-19|      39.93|     2.514|211.2891429|       8.106|    false|
|    1|2010-02-26|      46.63|     2.561|211.3196429|       8.106|    false|
|    1|2010-03-05|       46.5|     2.625|211.3501429|       8.106|    false|
+-----+----------+-----------+----------+-----------+------------+---------+
only showing top 5 rows

In [13]:
#join the train and test data with the store and features data
train = train_df.join(store_df, on='Store', how='left').join(feature_df, on=['Store', 'Date', 'IsHoliday'], how='left')
test = test_df.join(store_df, on='Store', how='left').join(feature_df, on=['Store', 'Date', 'IsHoliday'], how='left')

# Show the first few rows of the merged DataFrames (optional)
train.show(5)
test.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+----------+---------+----+------------+----+------+-----------+----------+-----------+------------+
|Store|      Date|IsHoliday|Dept|Weekly_Sales|Type|  Size|Temperature|Fuel_Price|        CPI|Unemployment|
+-----+----------+---------+----+------------+----+------+-----------+----------+-----------+------------+
|    1|2010-02-05|    false|   1|     24924.5|   A|151315|      42.31|     2.572|211.0963582|       8.106|
|    1|2010-02-12|     true|   1|    46039.49|   A|151315|      38.51|     2.548|211.2421698|       8.106|
|    1|2010-02-19|    false|   1|    41595.55|   A|151315|      39.93|     2.514|211.2891429|       8.106|
|    1|2010-02-26|    false|   1|    19403.54|   A|151315|      46.63|     2.561|211.3196429|       8.106|
|    1|2010-03-05|    false|   1|     21827.9|   A|151315|       46.5|     2.625|211.3501429|       8.106|
+-----+----------+---------+----+------------+----+------+-----------+----------+-----------+------------+
only showing top 5 rows

+-----+-----

In [14]:
#Replacing 'A','B','C' with 1,2,3 respectively in Type column
from pyspark.sql.functions import col, when
def maping_type(df):
    # map Type column
    df = df.withColumn('Type', when(col('Type') == 'A', '1')
                              .when(col('Type') == 'B', '2')
                              .when(col('Type') == 'C', '3')
                              .otherwise(col('Type')))
    return df

# Call the function and update the DataFrame
train=maping_type(train)
test=maping_type(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
from pyspark.sql import functions as F
#Extracting "week","month" and "year" using Date column
def extract_week(df):
    df = df.withColumn("week", F.weekofyear(df["Date"]))
    df = df.withColumn("month", F.month(df["Date"]))
    df = df.withColumn("year", F.year(df["Date"]))
    return df

train = extract_week(train)
test = extract_week(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType
from pyspark.sql.types import IntegerType


# Convert "CPI", "Type" and "Unemployment" columns to DoubleType
train = train.withColumn("CPI", train["CPI"].cast(DoubleType()))
train = train.withColumn("Unemployment", train["Unemployment"].cast(DoubleType()))

test = test.withColumn("CPI", test["CPI"].cast(DoubleType()))
test = test.withColumn("Unemployment", test["Unemployment"].cast(DoubleType()))

test = test.withColumn("Type", test["Type"].cast(IntegerType()))
train = train.withColumn("Type", train["Type"].cast(IntegerType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
#Storing modified train data into studio s3 bucket
output_path = "s3://sm-studio-bucket/SuperMarketSalesAnalysisOutput/train_output"

# Write DataFrame to CSV format
train.write.csv(output_path, mode='overwrite', header=True)

# Print confirmation message
print("DataFrame saved as CSV to S3:", output_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame saved as CSV to S3: s3://sm-studio-bucket/SuperMarketSalesAnalysisOutput/train_output

In [19]:
from pyspark.ml.regression import RandomForestRegressor, GBTRegressor, LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler

feature_columns = ['Store', 'IsHoliday', 'Dept', 'Type', 'Size', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'week', 'month', 'year']
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features", handleInvalid="keep")

train_data = assembler.transform(train)
test_data = assembler.transform(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
#Random Forest
rf1 = RandomForestRegressor(featuresCol="features", labelCol="Weekly_Sales", seed=42)
rf1_model = rf1.fit(train_data)

# Gradient Boosted Trees (GBT)
gbt = GBTRegressor(featuresCol="features", labelCol="Weekly_Sales", maxIter=10)
gbt_model = gbt.fit(train_data)

# Linear Regression
lr = LinearRegression(featuresCol="features", labelCol="Weekly_Sales")
lr_model = lr.fit(train_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# Step: Model Evaluation
evaluator = RegressionEvaluator(labelCol="Weekly_Sales", predictionCol="prediction", metricName="rmse")

# Evaluate Random Forest
rf1_predictions = rf1_model.transform(train_data)
rf1_rmse = evaluator.evaluate(rf1_predictions)
print("Random Forest RMSE on training data:", rf1_rmse)

# Evaluate Gradient Boosted Trees (GBT)
gbt_predictions = gbt_model.transform(train_data)
gbt_rmse = evaluator.evaluate(gbt_predictions)
print("GBT RMSE on training data:", gbt_rmse)

# Evaluate Linear Regression
lr_predictions = lr_model.transform(train_data)
lr_rmse = evaluator.evaluate(lr_predictions)
print("Linear Regression RMSE on training data:", lr_rmse)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Random Forest RMSE on training data: 17797.144800390153
GBT RMSE on training data: 14568.126646652017
Linear Regression RMSE on training data: 21704.703985619417

In [22]:
# Step: Model Selection
best_model = None
if rf1_rmse <= gbt_rmse and rf1_rmse <= lr_rmse:
    best_model = rf1_model
    print("Best Model :Random Forest", rf1_rmse)
elif gbt_rmse <= rf1_rmse and gbt_rmse <= lr_rmse:
    best_model = gbt_model
    print("Best Model :GBT", gbt_rmse)
else:
    best_model = lr_model
    print("Best Model :Linear Regression", lr_rmse)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Best Model :GBT 14568.126646652017

In [23]:
# Predict on test data using the best model
best_model_predictions = best_model.transform(test_data)
best_model_predictions.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+----------+---------+----+----+------+-----------+----------+-----------+------------+----+-----+----+--------------------+------------------+
|Store|      Date|IsHoliday|Dept|Type|  Size|Temperature|Fuel_Price|        CPI|Unemployment|week|month|year|            features|        prediction|
+-----+----------+---------+----+----+------+-----------+----------+-----------+------------+----+-----+----+--------------------+------------------+
|    1|2012-11-02|    false|   1|   1|151315|      55.32|     3.386|223.4627793|       6.573|  44|   11|2012|[1.0,0.0,1.0,1.0,...|33993.018829402834|
|    1|2012-11-09|    false|   1|   1|151315|      61.24|     3.314|223.4813073|       6.573|  45|   11|2012|[1.0,0.0,1.0,1.0,...|33993.018829402834|
|    1|2012-11-16|    false|   1|   1|151315|      52.92|     3.252|223.5129105|       6.573|  46|   11|2012|[1.0,0.0,1.0,1.0,...| 34453.29276501201|
|    1|2012-11-23|     true|   1|   1|151315|      56.23|     3.211|223.5619474|       6.573|  47|  

In [25]:
# Rename the "prediction" column to "Weekly_Sales_Prediction"
best_model_predictions = best_model_predictions.withColumnRenamed("prediction", "Weekly_Sales_Prediction")

# Dropping features vector column from pedicted dataframe
best_model_predictions = best_model_predictions.drop("features")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
best_model_predictions.printSchema()
best_model_predictions.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Store: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- IsHoliday: boolean (nullable = true)
 |-- Dept: integer (nullable = true)
 |-- Type: integer (nullable = true)
 |-- Size: integer (nullable = true)
 |-- Temperature: double (nullable = true)
 |-- Fuel_Price: double (nullable = true)
 |-- CPI: double (nullable = true)
 |-- Unemployment: double (nullable = true)
 |-- week: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- Weekly_Sales_Prediction: double (nullable = false)

+-----+----------+---------+----+----+------+-----------+----------+-----------+------------+----+-----+----+-----------------------+
|Store|      Date|IsHoliday|Dept|Type|  Size|Temperature|Fuel_Price|        CPI|Unemployment|week|month|year|Weekly_Sales_Prediction|
+-----+----------+---------+----+----+------+-----------+----------+-----------+------------+----+-----+----+-----------------------+
|    1|2012-11-02|    false|   1|  

In [27]:
output_path = "s3://sm-studio-bucket/SuperMarketSalesAnalysisOutput/Weekly_Sales_Prediction"

# Write DataFrame to CSV format
best_model_predictions.write.csv(output_path, mode='overwrite', header=True)

# Print confirmation message
print("DataFrame saved as CSV to S3:", output_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame saved as CSV to S3: s3://sm-studio-bucket/SuperMarketSalesAnalysisOutput/Weekly_Sales_Prediction

In [28]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
bm_feature_importance = best_model.featureImportances
# Calculating feature importance using best model
importance_data = [(feature, float(importance)) for feature, importance in zip(feature_columns, bm_feature_importance)]

# Define the schema with explicit data types
schema = StructType([
    StructField("feature", StringType(), True),
    StructField("importance", DoubleType(), True)
])

# Create the DataFrame with the defined schema
importance_df = spark.createDataFrame(importance_data, schema)

# Sort the DataFrame by importance in descending order
importance_df = importance_df.orderBy(col("importance").desc())
importance_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------------+
|     feature|          importance|
+------------+--------------------+
|        Dept|  0.5528327195116828|
|        Size| 0.23403628945486493|
|       Store| 0.11440820945889783|
|         CPI|0.052055876865314765|
|        week|0.021518288609701425|
|        Type|0.010311545266652839|
|Unemployment|0.008844330467123305|
|   IsHoliday|0.005167528294792795|
|        year|7.411211431310309E-4|
| Temperature| 7.61302973249225E-5|
|  Fuel_Price|7.960630513511024E-6|
|       month|                 0.0|
+------------+--------------------+

In [29]:
#Storing feature importance into studio s3 bucket
#S3 bucket output path
output_path = "s3://sm-studio-bucket/SuperMarketSalesAnalysisOutput/feature_importance"

# Write DataFrame to CSV format
importance_df.write.csv(output_path, mode='overwrite', header=True)

# Print confirmation message
print("DataFrame saved as CSV to S3:", output_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame saved as CSV to S3: s3://sm-studio-bucket/SuperMarketSalesAnalysisOutput/feature_importance